Reference: https://www.kaggle.com/code/ahmetsouksu/energy-efficiency-of-buildings-98-accuracy-r-2

In [ ]:
#libraries import
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
import math

In [5]:
df = pd.read_excel('ENB2012_data.xlsx')

df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    int64  
 6   X7      768 non-null    float64
 7   X8      768 non-null    int64  
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 60.1 KB


In [8]:
#checking for outliers

#z-score
from scipy import stats as st
z = np.abs(st.zscore(df))
print(np.where(z > 3))

(array([], dtype=int64), array([], dtype=int64))


In [9]:
#train-validation-test
#I will split the data into 3 as 60% train, 20% validation and 20% test.

train, validate, test = np.split(df.sample(frac=1, random_state=42),
                                [int(.6*len(df)), int(.8*len(df))])
print("train " , train)
print("validate " , validate)
print("test " , test)

train         X1     X2     X3      X4   X5  X6    X7  X8     Y1     Y2
668  0.62  808.5  367.5  220.50  3.5   2  0.40   3  16.47  16.90
324  0.66  759.5  318.5  220.50  3.5   2  0.25   1  13.17  16.39
624  0.98  514.5  294.0  110.25  7.0   2  0.40   3  32.82  32.78
690  0.79  637.0  343.0  147.00  7.0   4  0.40   4  41.32  46.23
473  0.64  784.0  343.0  220.50  3.5   3  0.25   4  16.69  19.76
..    ...    ...    ...     ...  ...  ..   ...  ..    ...    ...
190  0.62  808.5  367.5  220.50  3.5   4  0.10   3  12.71  14.14
115  0.79  637.0  343.0  147.00  7.0   5  0.10   2  36.03  42.86
732  0.82  612.5  318.5  147.00  7.0   2  0.40   5  30.00  29.93
467  0.69  735.0  294.0  220.50  3.5   5  0.25   4  12.86  16.13
94   0.62  808.5  367.5  220.50  3.5   4  0.10   1  12.93  14.33

[460 rows x 10 columns]
validate         X1     X2     X3      X4   X5  X6    X7  X8     Y1     Y2
180  0.66  759.5  318.5  220.50  3.5   2  0.10   3  11.59  13.46
301  0.82  612.5  318.5  147.00  7.0   3  0.25  

In [10]:
#now i split x columns and y columns
x_train = train.iloc[:,:-2]
y_train = train.iloc[:,8:]
x_valid = validate.iloc[:,:-2]
y_valid = validate.iloc[:,8:]
x_test = test.iloc[:,:-2]
y_test = test.iloc[:,8:]

In [11]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

In [12]:
def print_score(m):
    m.fit(x_train,y_train)
    
    print(f"R^2 of train set: {m.score(x_train, y_train)}")
    print(f"R^2 of validation set: {m.score(x_valid, y_valid)}")

In [13]:
m = RandomForestRegressor(n_estimators=10, n_jobs=-1, max_features = .5, max_leaf_nodes=10)
m.fit(x_train, y_train)
m.score(x_train, y_train)
m.score(x_valid, y_valid)

0.9622399800336123

In [14]:
#max leaf and features=0.5
m1_1 = RandomForestRegressor(n_estimators=10, n_jobs=-1, max_features = .5)
#max leaf and max features
m1_2 = RandomForestRegressor(n_estimators=10, n_jobs=-1)
#25 leaf and max faeatures
m1_3 = RandomForestRegressor(n_estimators=10, n_jobs=-1,max_leaf_nodes=25)
#50 leaf and features=0.75
m1_4 = RandomForestRegressor(n_estimators=10, n_jobs=-1, max_leaf_nodes=50
                            , max_features = .75)
#50 leaf and features=0.25
m1_5 = RandomForestRegressor(n_estimators=10, n_jobs=-1, max_leaf_nodes=100
                            , max_features = .25)

In [15]:
print_score(m1_1)

R^2 of train set: 0.9962161834452916
R^2 of validation set: 0.9766907307681628


In [16]:
print_score(m1_1)
print("*********************")
print_score(m1_2)
print("*********************")
print_score(m1_3)
print("*********************")
print_score(m1_4)
print("*********************")
print_score(m1_5)

R^2 of train set: 0.996346246393919
R^2 of validation set: 0.976360263828908
*********************
R^2 of train set: 0.995982713548573
R^2 of validation set: 0.9750675951906225
*********************
R^2 of train set: 0.982470833551617
R^2 of validation set: 0.978608368792188
*********************
R^2 of train set: 0.9901524238912385
R^2 of validation set: 0.9784422711583043
*********************
R^2 of train set: 0.9927082956674762
R^2 of validation set: 0.975907539428
